# pyrecorder

# Installation

The framework is available at the PyPi Repository:

# Usage

## GIF

In [6]:
import numpy as np
import matplotlib.pyplot as plt

from pyrecorder.video import Video
from pyrecorder.recorders.gif import GIF

vid = Video(GIF("example.gif"))

for k in range(10):
    X = np.random.random((100, 2))
    plt.scatter(X[:, 0], X[:, 1])
    vid.record()

vid.close()

In [11]:
%%HTML
<img src="example.gif" alt="ExampleGIF">

## Video

In [13]:
import numpy as np
import matplotlib.pyplot as plt

from pyrecorder.video import Video
from pyrecorder.recorders.file import File

vid = Video(File("example.mp4"))

for k in range(10):
    X = np.random.random((100, 2))
    plt.scatter(X[:, 0], X[:, 1])
    vid.record()

vid.close()

# Contact